# Refreshing DWH Dimension using Python

## Preparation

In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect(':memory:')

In [3]:
r = conn.cursor()

In [4]:
source_data = {
    'Customer_ID': [101, 102, 103],
    'Customer_Name': ['MVG', 'MVB', 'MVH'],
    'Location': ['München', 'Berlin', 'Hamburg']
}

In [5]:
r.execute("DROP TABLE IF EXISTS source_data_customer;")

In [6]:
source_df = pd.DataFrame(source_data)

In [7]:
source_df.to_sql('source_data_customer', conn)

In [8]:
# check data in table
pd.read_sql_query('SELECT * FROM source_data_customer', conn)

,index,Customer_ID,Customer_Name,Location
0,0,101,MVG,München
1,1,102,MVB,Berlin
2,2,103,MVH,Hamburg


## INIT load into DWH Dimension

In [9]:
# extend data set with start_date, end_date
# Business key and Surrogate key are the same regarding simplicity

In [10]:
df = pd.read_sql_query('SELECT * FROM source_data_customer', conn) # read from source table

In [11]:
start_date = pd.to_datetime('01.01.2021') # enrich data
end_date = pd.to_datetime('NaN') # enrich data

In [12]:
df['start_date'] = start_date # extend data frame with enriched data
df['end_date'] = end_date

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   index          3 non-null      int64         
 1   Customer_ID    3 non-null      int64         
 2   Customer_Name  3 non-null      object        
 3   Location       3 non-null      object        
 4   start_date     3 non-null      datetime64[ns]
 5   end_date       0 non-null      datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(2)
memory usage: 272.0+ bytes


In [14]:
r.execute('DROP TABLE IF EXISTS D_CUSTOMER;') # INIT load into DWH

In [15]:
df[['Customer_ID', 'start_date', 'end_date', 'Customer_Name', 'Location']].to_sql('D_CUSTOMER', conn) # INIT load into DWH

In [16]:
pd.read_sql_query('SELECT * FROM D_CUSTOMER', conn) # check the content of DWH Dimension

,index,Customer_ID,start_date,end_date,Customer_Name,Location
0,0,101,2021-01-01 00:00:00,None,MVG,München
1,1,102,2021-01-01 00:00:00,None,MVB,Berlin
2,2,103,2021-01-01 00:00:00,None,MVH,Hamburg


## Increment load into DWH Dimension

### Source data have been changed

In [17]:
# one customer had been updated
v_sql = "UPDATE source_data_customer set Customer_Name = ? WHERE Customer_ID = ?;"
values = ('BVG', '102')
r.execute(v_sql, values)

In [18]:
# another customer had been deleted
v_sql = "DELETE FROM source_data_customer WHERE Customer_ID = ?;"
values = ('103',)
r.execute(v_sql, values)

### Compare source and DWH data and refresh DWH Dimension

In [19]:
s_df = pd.read_sql_query('SELECT * FROM source_data_customer;', conn) # read updated source

In [20]:
s_df

,index,Customer_ID,Customer_Name,Location
0,0,101,MVG,München
1,1,102,BVG,Berlin


In [21]:
dwh_df = pd.read_sql_query('SELECT * FROM D_CUSTOMER', conn) # read the content of DWH Dimension

In [22]:
dwh_df

,index,Customer_ID,start_date,end_date,Customer_Name,Location
0,0,101,2021-01-01 00:00:00,None,MVG,München
1,1,102,2021-01-01 00:00:00,None,MVB,Berlin
2,2,103,2021-01-01 00:00:00,None,MVH,Hamburg


In [23]:
# check candidates for logical deletion
df_diff = pd.merge(dwh_df, s_df, how='left', on='Customer_ID', indicator='Exist')
df_diff = df_diff.loc[df_diff['Exist'] == 'left_only']

In [24]:
v_sql = 'UPDATE D_CUSTOMER SET end_date = ? WHERE Customer_ID = ?;'
values = (str(pd.to_datetime('today').normalize()), df_diff['Customer_ID'].item())
r.execute(v_sql, values)

In [25]:
# check candidates for update
# only attributes have been checked
df_diff  = pd.merge(dwh_df, s_df, on=["Customer_Name", "Location"], how='right', indicator='Exist')
df_diff  = df_diff.loc[df_diff['Exist'] == 'right_only']

In [26]:
df_diff

,index_x,Customer_ID_x,start_date,end_date,Customer_Name,Location,index_y,Customer_ID_y,Exist
1,NaN,NaN,NaN,NaN,BVG,Berlin,1,102,right_only


In [27]:
v_sql = 'UPDATE D_CUSTOMER SET Customer_Name = ? WHERE Customer_ID = ?;'
values = (df_diff['Customer_Name'].item(), df_diff['Customer_ID_y'].item())
r.execute(v_sql, values)

## Status of DWH Dimension D_CUSTOMER

In [28]:
pd.read_sql_query('SELECT * FROM D_CUSTOMER', conn)

,index,Customer_ID,start_date,end_date,Customer_Name,Location
0,0,101,2021-01-01 00:00:00,None,MVG,München
1,1,102,2021-01-01 00:00:00,None,BVG,Berlin
2,2,103,2021-01-01 00:00:00,2021-04-15 00:00:00,MVH,Hamburg


In [29]:
r.close()
conn.close()